In [10]:
import json
import math
import random
import torch

import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR
from torch.optim import Adam
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [11]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [12]:
def process_data(file_path):
    with open(file_path, "r") as file:
        datas = json.load(file)

    dataset = []
    for data in datas:
        user_info = []
        for _, value in data["user_info"].items():
            if value:
                user_info.append(value)
            else:
                user_info.append(0)
        
        parameter = []
        for _, value in data["parameter"].items():
            parameter.append(value)
        dataset.append({"user_info": user_info, "parameter": parameter})

    return dataset

In [13]:
from torch.utils.data import DataLoader, TensorDataset
train_path = "../../train_data/type_1/train.json"
valid_path = "../../train_data/type_1/valid.json"

train_dataset = process_data(train_path)
valid_dataset = process_data(valid_path)

user_infos = [for_data["user_info"] for for_data in train_dataset]
parameters = [for_data["parameter"] for for_data in train_dataset]

user_infos = np.array(user_infos)
parameters = np.array(parameters)

# 将输入序列中的索引值映射到0到60之间
# user_infos = user_infos % 61
# 将缺失值替换为特定值，例如：60
user_infos[user_infos == 0] = 1000

user_infos = torch.tensor(user_infos, dtype=torch.long)
parameters = torch.tensor(parameters, dtype=torch.float32)

dataset = TensorDataset(user_infos, parameters)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)


for user_info, parameter in train_loader:
    print(user_info)
    print(parameter)
    break


tensor([[  52,   80, 1000,  ..., 1000, 1000, 1000],
        [  71, 1000, 1000,  ..., 1000, 1000, 1000],
        [  71,  100,   85,  ..., 1000, 1000, 1000],
        ...,
        [  43,  100, 1000,  ..., 1000, 1000, 1000],
        [1000,   90, 1000,  ..., 1000, 1000, 1000],
        [  70,   95, 1000,  ..., 1000, 1000, 1000]])
tensor([[ 6., 48., 48.,  ..., 12.,  6.,  3.],
        [ 2., 29., 22.,  ..., 10.,  6.,  3.],
        [ 3., 23., 34.,  ..., 12.,  6.,  3.],
        ...,
        [ 0., 24., 38.,  ..., 10.,  3.,  9.],
        [ 4., 40., 21.,  ..., 10.,  6.,  3.],
        [ 9., 43., 41.,  ..., 12.,  6.,  3.]])


In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # Shape (1, max_len, d_model)
        self.register_buffer('pe', pe)


    def forward(self, x):
        """
            x: (batch_size, x_len, d_model)
            requires_grad_(False) is used to prevent the model from updating the positional encoding
        """
        x = x + self.pe[:, : x.size(1), :].requires_grad_(False)
        return self.dropout(x)

In [15]:
class TransformerMultiOutputRegressor(nn.Module):
    def __init__(self, n_inputs, n_outputs, d_model, n_heads, n_layers, dim_feedforward=256, dropout=0.1, batch_first=True):
        super().__init__()
        self.embedding = nn.Embedding(n_inputs, d_model, padding_idx=0)
        encode_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=batch_first)
        self.transformer_encoder = nn.TransformerEncoder(encode_layer, num_layers=n_layers)
        self.positional_encoding = PositionalEncoding(d_model=d_model, dropout=dropout)
        self.regressor = nn.Linear(d_model, n_outputs)


    def forward(self, src):
        """
            src: (batch_size, src_len)
        """
        src = self.embedding(src)
        src = self.positional_encoding(src)
        src = self.transformer_encoder(src)
        output = self.regressor(src) # (batch_size, seq_len, output_dim)
        output = torch.mean(output, dim=1) # (batch_size, output_dim)
        return output

In [16]:
# 初始化模型
n_inputs = 1001
n_outputs = 119
d_model = 32
n_heads = 8
n_layers = 6
dim_feedforward = 256
dropout = 0.1
batch_first = True

model = TransformerMultiOutputRegressor(n_inputs, n_outputs, d_model, n_heads, n_layers, dim_feedforward, dropout, batch_first)
model

TransformerMultiOutputRegressor(
  (embedding): Embedding(1001, 32, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (regressor): Linear(in_features=32, out_features=119, bias=True)
)

In [17]:
# Model, loss function, optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


IndexError: index out of range in self